In [1]:
import keras
import csv
import scipy
import numpy as np
import nltk #from nltk.tokenize import punkt #.tokenize.moses import MosesTokenizer
from gensim.models import KeyedVectors
from itertools import islice
import pickle
import pandas as pd
import re
from nltk.corpus import stopwords
import json
import pickle
from cyordereddict import OrderedDict
from joblib import Parallel, delayed
pattern = re.compile(r'[^\s\w_]+')

DATASET_FILE_PATH = "./DataSet/yelp.csv/yelp.csv"
WORD2VEC_FILE_PATH = "./DataSet/FBword2vec/wiki.en.vec"
TRAINING_DATA_PATH = "./DataSet/trainingData"
EXTENDED_DATASET_PATH = "./DataSet/review.json"

###File Names
TEST_NAME = "standard_tokenizer_No_Stopwords_noPunctuation_FB_vectors_100percentDataset"

EMBEDDED_MATRIX_NAME = TEST_NAME+'_embedded_matrix.pkl'
TOKENIZED_SENTENCES_NAME = TEST_NAME+ "_tokenized_sentences.pkl"
WORD_INDEX_NAME = TEST_NAME+ "_word_index.pkl"
INDEXED_TOKENIZED_SENTENCES_NAME = TEST_NAME + "_indexed_tokenized_sentences.pkl"

LABELS_NAME = 'labels.pkl'


AMOUNT_OF_WORDS = 70
FACEBOOK_EMBEDDING_SIZE = 300

C:\Users\skymont\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\skymont\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
word_index = pickle.load(open(WORD_INDEX_NAME,'rb'))

In [3]:
embedded_matrix =  np.load(EMBEDDED_MATRIX_NAME+'.npy')
print('embedded matrix loaded')

embedded matrix loaded


In [4]:
indexed_tokenized_sentences = pickle.load(open(INDEXED_TOKENIZED_SENTENCES_NAME,'rb'))

In [5]:
def reshapeSentences(indexed_tokenized_sentences,sentence_length):
    for i in range(len(indexed_tokenized_sentences)):
        padding = sentence_length - len(indexed_tokenized_sentences[i])
        indexed_tokenized_sentences[i] = indexed_tokenized_sentences[i][:sentence_length] + [-1]*padding
    return indexed_tokenized_sentences

In [6]:
### BUILDING MODEL

In [7]:
reshaped_Sentences = reshapeSentences(indexed_tokenized_sentences, AMOUNT_OF_WORDS)

In [8]:
labels = pd.read_pickle(LABELS_NAME)

In [9]:
labels.head()
labels.pop('cool')
labels.pop('funny')
labels.pop('useful')
labels.head()

,stars
0,5
1,5
2,5
3,5
4,4


In [10]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
import numpy as np
from keras import backend
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [11]:
embedding_layer = Embedding(
    len(word_index),
    FACEBOOK_EMBEDDING_SIZE,
    weights=[embedded_matrix],
    input_length=AMOUNT_OF_WORDS,
    trainable=True
)

In [12]:
reshaped_Sentences = np.stack(reshaped_Sentences,axis=0)

In [13]:

X_train, X_test, y_train, y_test = train_test_split(reshaped_Sentences, labels.values, test_size=0.2, random_state=42)

In [14]:
def generator(features, labels, batch_size, replace=True):
    
    batch_features = np.zeros((batch_size,AMOUNT_OF_WORDS))
    batch_labels = np.zeros((batch_size,1))
    while True:
        index = np.random.choice(features.shape[0], batch_size, replace=replace)
        batch_features[:] = features[index]
        batch_labels[:] = labels[index]
        yield batch_features, batch_labels
        

In [15]:
def precisionMetric(y_true,y_pred):
    #return tf.metrics.mean(y_true - tf.round(y_pred))[0]
    return backend.mean(y_true - backend.round(y_pred))

In [16]:
def percentageCorrect(y_true, y_pred):
    return backend.mean(backend.equal(y_true, backend.round(y_pred)))*100

In [17]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.25))
model.add(Conv1D(32, 5, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=5))
model.add(LSTM(70))
model.add(Dense(200))
model.add(Dropout(0.8))
model.add(Dense(200))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[percentageCorrect])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 70, 300)           57183900  
_________________________________________________________________
dropout_1 (Dropout)          (None, 70, 300)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 66, 32)            48032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 70)                28840     
_________________________________________________________________
dense_1 (Dense)              (None, 200)               14200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
__________

In [18]:
#model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=40, batch_size=10, verbose=2)
model.fit_generator(generator(X_train,y_train,8000,False),steps_per_epoch=100, epochs=200, verbose=2, validation_data=generator(X_test, y_test, 15000, False), validation_steps=5)

Epoch 1/200
 - 36s - loss: 2.7776 - percentageCorrect: 23.0868 - val_loss: 0.8495 - val_percentageCorrect: 44.9293
Epoch 2/200
 - 27s - loss: 0.9982 - percentageCorrect: 39.8425 - val_loss: 0.6988 - val_percentageCorrect: 50.7027
Epoch 3/200
 - 27s - loss: 0.8822 - percentageCorrect: 43.1265 - val_loss: 0.6375 - val_percentageCorrect: 55.7853
Epoch 4/200
 - 27s - loss: 0.8013 - percentageCorrect: 45.8821 - val_loss: 0.6394 - val_percentageCorrect: 54.0600
Epoch 5/200
 - 27s - loss: 0.7281 - percentageCorrect: 48.6166 - val_loss: 0.6143 - val_percentageCorrect: 55.7907
Epoch 6/200
 - 27s - loss: 0.6696 - percentageCorrect: 51.3804 - val_loss: 0.5838 - val_percentageCorrect: 57.7080
Epoch 7/200
 - 27s - loss: 0.6284 - percentageCorrect: 53.8046 - val_loss: 0.5801 - val_percentageCorrect: 58.5907
Epoch 8/200
 - 27s - loss: 0.6121 - percentageCorrect: 55.1539 - val_loss: 1.4110 - val_percentageCorrect: 50.3240
Epoch 9/200
 - 27s - loss: 0.5898 - percentageCorrect: 56.4310 - val_loss: 0.573

Epoch 72/200
 - 27s - loss: 0.4015 - percentageCorrect: 63.8914 - val_loss: 0.5128 - val_percentageCorrect: 63.8947
Epoch 73/200
 - 27s - loss: 0.4029 - percentageCorrect: 63.8881 - val_loss: 0.5163 - val_percentageCorrect: 64.0547
Epoch 74/200
 - 27s - loss: 0.3966 - percentageCorrect: 64.1316 - val_loss: 0.5126 - val_percentageCorrect: 63.3840
Epoch 76/200
 - 27s - loss: 0.3969 - percentageCorrect: 64.0414 - val_loss: 0.5112 - val_percentageCorrect: 63.5307
Epoch 77/200
 - 27s - loss: 0.3930 - percentageCorrect: 64.2367 - val_loss: 0.5135 - val_percentageCorrect: 63.7813
Epoch 78/200
 - 27s - loss: 0.3935 - percentageCorrect: 64.2979 - val_loss: 0.5242 - val_percentageCorrect: 63.9653
Epoch 79/200
 - 27s - loss: 0.3922 - percentageCorrect: 64.2755 - val_loss: 0.5151 - val_percentageCorrect: 64.4200
Epoch 80/200
 - 27s - loss: 0.3900 - percentageCorrect: 64.4079 - val_loss: 0.5121 - val_percentageCorrect: 63.8987
Epoch 81/200
 - 27s - loss: 0.3900 - percentageCorrect: 64.5475 - val_lo

Epoch 144/200
 - 27s - loss: 0.3348 - percentageCorrect: 67.2186 - val_loss: 0.5326 - val_percentageCorrect: 63.9320
Epoch 145/200
 - 27s - loss: 0.3343 - percentageCorrect: 67.2735 - val_loss: 0.5297 - val_percentageCorrect: 63.9613
Epoch 146/200
 - 27s - loss: 0.3345 - percentageCorrect: 67.1913 - val_loss: 0.5377 - val_percentageCorrect: 64.2547
Epoch 147/200
 - 27s - loss: 0.3322 - percentageCorrect: 67.3370 - val_loss: 0.5376 - val_percentageCorrect: 63.9720
Epoch 148/200
 - 27s - loss: 0.3330 - percentageCorrect: 67.3210 - val_loss: 0.5343 - val_percentageCorrect: 64.2400
Epoch 149/200
 - 27s - loss: 0.3326 - percentageCorrect: 67.3684 - val_loss: 0.5261 - val_percentageCorrect: 64.2693
Epoch 150/200
 - 27s - loss: 0.3307 - percentageCorrect: 67.4980 - val_loss: 0.5380 - val_percentageCorrect: 64.3093
Epoch 151/200
 - 27s - loss: 0.3287 - percentageCorrect: 67.5344 - val_loss: 0.5373 - val_percentageCorrect: 63.8613
Epoch 152/200
 - 27s - loss: 0.3310 - percentageCorrect: 67.4198